In [1]:
from datasets import load_dataset
import json
from datasets import load_dataset

tokenizer = {char:i for i, char in enumerate(".123456789")}

# Load Sudoku dataset
ds = load_dataset("sapientinc/sudoku-extreme")

# Make your tokenizer (char → id)
tokenizer = {char:i for i, char in enumerate(".123456789")}

# Function to tokenize one row
def tokenize_example(example):
    puzzle = example["question"]
    solution = example["answer"]

    # Convert chars to integer IDs
    puzzle_ids = [tokenizer[c] for c in puzzle]
    solution_ids = [tokenizer[c] for c in solution]

    return {
        "puzzle_ids": puzzle_ids,
        "solution_ids": solution_ids
    }

# Apply transformation
ds = ds.map(tokenize_example)

print(ds)
print(ds["train"][0])


Map:   0%|          | 0/3831994 [00:00<?, ? examples/s]

Map:   0%|          | 0/422786 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'question', 'answer', 'rating', 'puzzle_ids', 'solution_ids'],
        num_rows: 3831994
    })
    test: Dataset({
        features: ['source', 'question', 'answer', 'rating', 'puzzle_ids', 'solution_ids'],
        num_rows: 422786
    })
})
{'source': 'puzzles4_forum_hardest_1905', 'question': '5...27..9..41......1..5.3...92.6.8...5......66..7..29.8...7...2.......8...9..36..', 'answer': '583427169974136528216859374792364851351298746648715293865971432137642985429583617', 'rating': 18, 'puzzle_ids': [5, 0, 0, 0, 2, 7, 0, 0, 9, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 3, 0, 0, 0, 9, 2, 0, 6, 0, 8, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 7, 0, 0, 2, 9, 0, 8, 0, 0, 0, 7, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 9, 0, 0, 3, 6, 0, 0], 'solution_ids': [5, 8, 3, 4, 2, 7, 1, 6, 9, 9, 7, 4, 1, 3, 6, 5, 2, 8, 2, 1, 6, 8, 5, 9, 3, 7, 4, 7, 9, 2, 3, 6, 4, 8, 5, 1, 3, 5, 1, 2, 9, 8, 7, 4, 6, 6, 4, 8, 7, 1, 5, 2, 9, 3, 8, 6, 5

In [9]:
from tqdm import tqdm

train_data = []
for data in tqdm(ds['train'], "Processing"):
    train_data.append(data['solution_ids'])
    
print(len(train_data))

Processing: 100%|██████████| 3831994/3831994 [12:50<00:00, 4972.30it/s]

3831994


In [6]:
import torch

In [10]:
t = torch.tensor(train_data, dtype=torch.int8)
torch.save(t, "Label_Data.pt")

In [11]:
del train_data

In [8]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader, TensorDataset
import json
from datasets import load_dataset

torch.set_default_device('cuda')

# Your HRM model (paste your simplified model here)
from Model import HierarchicalReasoningModel_ACTV1

# sudoku_dataset = TensorDataset(train_data[:train_data.shape[0]-1000], label_data[:train_data.shape[0]-1000])
# test_dataset = TensorDataset(train_data[train_data.shape[0]-1000:], label_data[train_data.shape[0]-1000:])


class SudokuDataset(Dataset):
    def __init__(self, split='train'):
        self.tokenizer = {char: i for i, char in enumerate(".123456789")}
        
        # self.train_data = torch.load("Data.pt")[:3831994-2000].to(torch.int16)
        # self.label_data = torch.load("Label_Data.pt")[:3831994-2000].to(torch.int16)
        
        self.train_data = torch.load("Data.pt", map_location='cpu')[3831994-2000:].long()
        self.label_data = torch.load("Label_Data.pt", map_location='cpu')[3831994-2000:].long()
        
        # # Load tokenized questions
        # with open(data_file, 'r') as f:
        #     self.questions = [json.loads(line) for line in f]
        
        # # Get answers from original dataset
        # self.answers = []
        # for i in range(len(ds[split])):
        #     tokenized_answer = [self.tokenizer[char] for char in ds[split][i]['answer']]
        #     self.answers.append(tokenized_answer)
    

    
    def __len__(self):
        return self.train_data.shape[0]
    
    def __getitem__(self, idx):
        return {
            'inputs': self.train_data[idx],
            'targets': self.label_data[idx],
            'puzzle_identifiers': torch.tensor([0], dtype=torch.long)  # Single puzzle type for now
        }
        

class SudokuDataset2(Dataset):
    def __init__(self, split='train'):
        self.tokenizer = {char: i for i, char in enumerate(".123456789")}
        
        # self.train_data = torch.load("Data.pt")[3831994-2000:].to(torch.int16)
        # self.label_data = torch.load("Label_Data.pt")[3831994-2000:].to(torch.int16)
        
        self.train_data = torch.load("Data.pt", map_location='cpu')[3831994-2000:].long()
        self.label_data = torch.load("Label_Data.pt", map_location='cpu')[3831994-2000:].long()
        # # Load tokenized questions
        # with open(data_file, 'r') as f:
        #     self.questions = [json.loads(line) for line in f]
        
        # # Get answers from original dataset
        # self.answers = []
        # for i in range(len(ds[split])):
        #     tokenized_answer = [self.tokenizer[char] for char in ds[split][i]['answer']]
        #     self.answers.append(tokenized_answer)
    

    
    def __len__(self):
        return self.train_data.shape[0]
    
    def __getitem__(self, idx):
        return {
            'inputs': self.train_data[idx],
            'targets': self.label_data[idx],
            'puzzle_identifiers': torch.tensor([0], dtype=torch.long)  # Single puzzle type for now
        }

class HRMLightning(pl.LightningModule):
    def __init__(self, lr=1e-4):
        super().__init__()
        
        # HRM config
        config = {
            'batch_size': 8,
            'seq_len':81,  # 9x9 sudoku
            'vocab_size': 10,  # 0-9 digits
            'num_puzzle_identifiers': 1,
            'hidden_size': 256,
            'num_heads': 8,
            'H_layers': 2,
            'L_layers': 2,
            'H_cycles': 2,
            'L_cycles': 4,
            'halt_max_steps': 5
        }
        
        self.model = HierarchicalReasoningModel_ACTV1(config)
        self.lr = lr
        self.carry = None  # HRM state
    
    def forward(self, batch):
        # Initialize carry if None
        if self.carry is None:
            self.carry = self.model.initial_carry(batch)
        
        # HRM forward with carry state
        self.carry, outputs = self.model(self.carry, batch)
        return outputs
    
    def training_step(self, batch, batch_idx):
        # Reset carry periodically (every 10 steps)
        if batch_idx % 10 == 0:
            self.carry = None
            
        outputs = self(batch)
        
        # Language modeling loss
        logits = outputs['logits']  # [batch, seq, vocab]
        targets = batch['targets']   # [batch, seq]
        
        loss = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)), 
            targets.reshape(-1)
        )
        
        # ACT loss (if available)
        if 'q_halt_logits' in outputs and 'target_q_continue' in outputs:
            q_loss = F.mse_loss(
                torch.sigmoid(outputs['q_continue_logits']), 
                outputs['target_q_continue']
            )
            loss = loss + 0.1 * q_loss
        
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        self.carry = None  # Reset for clean validation
        outputs = self(batch)
        
        logits = outputs['logits']
        targets = batch['targets']
        
        loss = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)), 
            targets.reshape(-1)
        )
        
        # Accuracy
        preds = logits.argmax(-1)
        acc = (preds == targets).float().mean()
        
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

# Training script
def train_hrm_sudoku():
    # Load dataset
    ds = load_dataset("sapientinc/sudoku-extreme")
    
    # Create datasets
    train_dataset = SudokuDataset('train')
    val_dataset = SudokuDataset2('test')  # You'll need val data too
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)
    
    # Model
    model = HRMLightning(lr=1e-4)
    print("Initialized")
    # Trainer
    trainer = pl.Trainer(
        max_epochs=10,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        precision='16-mixed',  # Faster training
        val_check_interval=0.25,  # Validate 4 times per epoch
        log_every_n_steps=1,
    )
    print("here")
    # Train
    trainer.fit(model, train_loader, val_loader)
    print("and here")

if __name__ == "__main__":
    train_hrm_sudoku()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                             | Params | Mode 
-------------------------------------------------------------------
0 | model | HierarchicalReasoningModel_ACTV1 | 1.9 M  | train
-------------------------------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.448     Total estimated model params size (MB)
39        Modules in train mode
0         Modules in eval mode


Initialized
here


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.